In [179]:
import pandas as pd
import statsmodels.api as sm
import plotly.graph_objects as go
import statsmodels
from stargazer.stargazer import Stargazer
import statsmodels.formula.api as smf

In [180]:
df = pd.read_stata("../data/variables_productos_primarios.dta",index_col="fecha")
df = df.drop(["mes","anio"],axis=1)
df["pp"] = df.pp / 1_000_000

In [181]:
# Descomposición de las series
decomposition_pp = sm.tsa.seasonal_decompose(df['pp'], model='additive', two_sided=False)
df['pp_tendencia'] = decomposition_pp.trend
df['pp_estacional'] = decomposition_pp.seasonal
def calcula_serie_estacionaria(df: pd.DataFrame, variables_a_estacionarizar: list[str]):
    '''Quitamos tendencia y ciclo de las series'''
    for variable in variables_a_estacionarizar:
        decomposition= sm.tsa.seasonal_decompose(df[variable], model='additive', two_sided=False)
        df[f"{variable}_estacionaria"] = decomposition.resid
    return df
df = calcula_serie_estacionaria(df, variables_a_estacionarizar=["itcr","ip", "pre"])
# df = df.drop(["itcr", "ip", "pre"], axis=1)
df

,itcr,ip,pre,pp,pp_tendencia,pp_estacional,itcr_estacionaria,ip_estacionaria,pre_estacionaria
fecha,,,,,,,,,
2005-01-01,105.929359,86.866026,1423.333333,508.020737,NaN,-219.379456,NaN,NaN,NaN
2005-02-01,103.960244,87.907449,969.666667,500.539941,NaN,-282.817487,NaN,NaN,NaN
2005-03-01,102.549715,86.226525,1730.000000,656.181802,NaN,-69.992233,NaN,NaN,NaN
2005-04-01,101.267876,87.964450,620.000000,939.064907,NaN,233.736746,NaN,NaN,NaN
2005-05-01,100.294069,89.117152,54.666667,1028.823820,NaN,394.916431,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2022-07-01,44.102487,208.916689,0.000000,2431.418408,2049.265446,292.309439,-5.302398,12.056387,61.461167
2022-08-01,43.439754,200.965695,146.666667,1988.408046,2019.815046,294.666613,-5.571195,4.974320,178.901527
2022-09-01,43.289343,205.021010,106.666667,2071.879245,1978.316012,-5.483544,-5.364569,10.197078,-180.222656


In [182]:
def create_lag_columns(df:pd.DataFrame, variable_name:str, max_lag:int):
    for lag in range(1, max_lag + 1):
        new_column_name = f"{variable_name}_lag_{lag}"
        df[new_column_name] = df[variable_name].shift(lag)

In [183]:
create_lag_columns(df, "itcr_estacionaria", 9)
create_lag_columns(df, "ip_estacionaria", 9)
create_lag_columns(df, "pre_estacionaria", 9)
create_lag_columns(df, "pp", 9)
create_lag_columns(df, "pp_tendencia", 6)
create_lag_columns(df, "pp_estacional", 6)

In [184]:
df

,itcr,ip,pre,pp,pp_tendencia,pp_estacional,itcr_estacionaria,ip_estacionaria,pre_estacionaria,itcr_estacionaria_lag_1,...,pp_tendencia_lag_3,pp_tendencia_lag_4,pp_tendencia_lag_5,pp_tendencia_lag_6,pp_estacional_lag_1,pp_estacional_lag_2,pp_estacional_lag_3,pp_estacional_lag_4,pp_estacional_lag_5,pp_estacional_lag_6
fecha,,,,,,,,,,,,,,,,,,,,,
2005-01-01,105.929359,86.866026,1423.333333,508.020737,NaN,-219.379456,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-02-01,103.960244,87.907449,969.666667,500.539941,NaN,-282.817487,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-219.379456,NaN,NaN,NaN,NaN,NaN
2005-03-01,102.549715,86.226525,1730.000000,656.181802,NaN,-69.992233,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-282.817487,-219.379456,NaN,NaN,NaN,NaN
2005-04-01,101.267876,87.964450,620.000000,939.064907,NaN,233.736746,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-69.992233,-282.817487,-219.379456,NaN,NaN,NaN
2005-05-01,100.294069,89.117152,54.666667,1028.823820,NaN,394.916431,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,233.736746,-69.992233,-282.817487,-219.379456,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-01,44.102487,208.916689,0.000000,2431.418408,2049.265446,292.309439,-5.302398,12.056387,61.461167,-5.323903,...,2012.574120,1970.232076,1916.194531,1851.105363,258.043045,394.916431,233.736746,-69.992233,-282.817487,-219.379456
2022-08-01,43.439754,200.965695,146.666667,1988.408046,2019.815046,294.666613,-5.571195,4.974320,178.901527,-5.302398,...,2038.637362,2012.574120,1970.232076,1916.194531,292.309439,258.043045,394.916431,233.736746,-69.992233,-282.817487
2022-09-01,43.289343,205.021010,106.666667,2071.879245,1978.316012,-5.483544,-5.364569,10.197078,-180.222656,-5.571195,...,2045.668402,2038.637362,2012.574120,1970.232076,294.666613,292.309439,258.043045,394.916431,233.736746,-69.992233


In [169]:
fig = go.Figure(go.Scatter(x = df.index, y = df.pre_estacionaria, name = "t"))
fig.add_trace(go.Scatter(x=df.index, y = df.pre_estacionaria_lag_1, name = "t-1"))
fig.update_layout(template = None, title_text = "Prueba de rezagos")

In [170]:
df = df.dropna()
df

,itcr,ip,pre,pp,pp_tendencia,pp_estacional,itcr_estacionaria,ip_estacionaria,pre_estacionaria,itcr_estacionaria_lag_1,...,pp_tendencia_lag_3,pp_tendencia_lag_4,pp_tendencia_lag_5,pp_tendencia_lag_6,pp_estacional_lag_1,pp_estacional_lag_2,pp_estacional_lag_3,pp_estacional_lag_4,pp_estacional_lag_5,pp_estacional_lag_6
fecha,,,,,,,,,,,,,,,,,,,,,
2006-10-01,93.169667,114.377785,1593.333333,709.996617,682.417015,-140.237161,-3.106834,13.277998,583.084533,-3.066003,...,673.726938,683.379008,689.141547,692.347483,-5.483544,294.666613,292.309439,258.043045,394.916431,233.736746
2006-11-01,91.815703,126.158611,915.000000,685.784367,707.368368,-420.935943,-3.723575,20.340774,-89.011055,-3.106834,...,666.874379,673.726938,683.379008,689.141547,-140.237161,-5.483544,294.666613,292.309439,258.043045,394.916431
2006-12-01,90.468583,127.563604,1975.666667,686.993642,719.771294,-334.826450,-4.785537,15.810749,776.042356,-3.723575,...,669.666758,666.874379,673.726938,683.379008,-420.935943,-140.237161,-5.483544,294.666613,292.309439,258.043045
2007-01-01,90.065068,130.170429,813.333333,608.585972,720.150320,-219.379456,-4.889422,18.056411,-437.621349,-4.785537,...,682.417015,669.666758,666.874379,673.726938,-334.826450,-420.935943,-140.237161,-5.483544,294.666613,292.309439
2007-02-01,89.739941,131.133328,1903.000000,640.862915,724.626674,-282.817487,-4.802999,13.687013,545.981592,-4.889422,...,707.368368,682.417015,669.666758,666.874379,-219.379456,-334.826450,-420.935943,-140.237161,-5.483544,294.666613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-01,44.102487,208.916689,0.000000,2431.418408,2049.265446,292.309439,-5.302398,12.056387,61.461167,-5.323903,...,2012.574120,1970.232076,1916.194531,1851.105363,258.043045,394.916431,233.736746,-69.992233,-282.817487,-219.379456
2022-08-01,43.439754,200.965695,146.666667,1988.408046,2019.815046,294.666613,-5.571195,4.974320,178.901527,-5.302398,...,2038.637362,2012.574120,1970.232076,1916.194531,292.309439,258.043045,394.916431,233.736746,-69.992233,-282.817487
2022-09-01,43.289343,205.021010,106.666667,2071.879245,1978.316012,-5.483544,-5.364569,10.197078,-180.222656,-5.571195,...,2045.668402,2038.637362,2012.574120,1970.232076,294.666613,292.309439,258.043045,394.916431,233.736746,-69.992233


Hacer una tabla donde se vean los rezagos incorporados y el aic que tiene la regresión para cada uno de los rezagos

In [171]:
#Separar la base entre entrenamiento y testeo para evitar 'overfitting'
training = df.iloc[:-6].copy()
test = df.copy()

In [172]:
def plot_elecc_lag_variable(variable:str):
    X_list = []
    model_list = []
    lags = []

    for lag in range(1, 10):
        exog = [f"{variable}_lag_{i}" for i in range(1, lag + 1)]
        X = sm.add_constant(training[[*exog, "pp_tendencia", "pp_estacional"]])
        X_list.append(X)
        model = sm.OLS(training["pp"], X).fit()
        model_list.append(model)
        lags.append(lag)

    aic_values = [model.aic for model in model_list]

    fig = go.Figure(data=[go.Bar(x=lags, y=aic_values)])
    fig.update_yaxes(range = [(min(aic_values)*.999), max(aic_values)*1.001])
    fig.update_layout(
        template = None,
        title=f"AIC vs Lag {variable}",
        xaxis_title="Lag",
        yaxis_title="AIC",
        xaxis=dict(type='category'),  # Treat x-values as categories
    )
    return fig


In [173]:
exogenas = ["pp","pre_estacionaria","itcr_estacionaria", "ip_estacionaria"]
for exogena in exogenas:
    plot_elecc_lag_variable(exogena).show()
#PP: 5
#PRE: 7
#ITCR: 2
#IP: 4

Hacer una regresión con esos rezagos, incluyendo a la estacionalidad y tendencia. Con eso predigo al mes siguiente
Rezagos elegidos:
- PP: 5
- PRE: 7
- ITCR: 2
- IP: 4

In [174]:
exog = ["pre_estacionaria_lag_1", "pre_estacionaria_lag_2", "pre_estacionaria_lag_3", "pre_estacionaria_lag_4", "pre_estacionaria_lag_5","pre_estacionaria_lag_6","pre_estacionaria_lag_7",
        "itcr_estacionaria_lag_1", "itcr_estacionaria_lag_2",
        "ip_estacionaria_lag_1", "ip_estacionaria_lag_2", "ip_estacionaria_lag_3", "ip_estacionaria_lag_4",
        "pp_lag_1", "pp_lag_2", "pp_lag_3", "pp_lag_4", "pp_lag_5",
        "pp_estacional_lag_1", "pp_tendencia_lag_2"]

X = sm.add_constant(training[exog])
model = sm.RLM(training["pp"], X, M=sm.robust.norms.HuberT())
robust_results = model.fit()
robust_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:                     pp   No. Observations:                  188
Model:                            RLM   Df Residuals:                      167
Method:                          IRLS   Df Model:                           20
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Mon, 02 Oct 2023                                         
Time:                        16:58:43                                         
No. Iterations:                    11                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     143.8209     90.644      1.587      0.113     -33.839     321.480
pre_estacionaria_lag_1     -0.0325      0.034     -0.956      0.339      -0.099       0.034
pre_estacionaria_lag_2     -0.0378      0.034     -1.123      0.261      -0.104       0.028
pre_estacionaria_lag_3     -0.0216      0.034     -0.625      0.532      -0.089       0.046
pre_estacionaria_lag_4      0.0314      0.034      0.925      0.355      -0.035       0.098
pre_estacionaria_lag_5      0.0271      0.034      0.785      0.432      -0.041       0.095
pre_estacionaria_lag_6      0.0621      0.034      1.801      0.072      -0.005       0.130
pre_estacionaria_lag_7      0.0019      0.036      0.052      0.958      -0.068       0.072
itcr_estacionaria_lag_1     3.5220     10.952      0.322      0.748     -17.944      24.988
itcr_estacionaria_lag_2    -6.4701     10.744     -0.602      0.547     -27.527      14.587
ip_estacionaria_lag_1       1.3574      2.588      0.524      0.600      -3.715       6.430
ip_estacionaria_lag_2      -0.3160      3.484     -0.091      0.928      -7.145       6.513
ip_estacionaria_lag_3       1.4632      3.505      0.417      0.676      -5.406       8.332
ip_estacionaria_lag_4       2.1880      2.570      0.851      0.395      -2.848       7.224
pp_lag_1                    0.8146      0.084      9.720      0.000       0.650       0.979
pp_lag_2                   -0.1050      0.093     -1.127      0.260      -0.288       0.078
pp_lag_3                   -0.0172      0.093     -0.184      0.854      -0.200       0.166
pp_lag_4                   -0.0326      0.093     -0.351      0.726      -0.215       0.150
pp_lag_5                   -0.3165      0.078     -4.052      0.000      -0.470      -0.163
pp_estacional_lag_1        -0.1079      0.102     -1.056      0.291      -0.308       0.092
pp_tendencia_lag_2          0.5450      0.102      5.335      0.000       0.345       0.745
===========================================================================================

If the model instance has been used for another fit with different fit parameters, then the fit options might not be the correct ones anymore .
"""

In [175]:
from sklearn.metrics import mean_squared_error
from pandas.tseries.offsets import DateOffset


X_predict = sm.add_constant(training[exog])
predictions = robust_results.predict(X_predict)
rmse = mean_squared_error(training["pp"], predictions, squared=False)
# predictions.index = predictions.index + DateOffset(months=-1)
print("RMSE: ", rmse)
fig = go.Figure(go.Scatter(x = training.index, y = training["pp"], name = "observado"))
fig = fig.add_trace(go.Scatter(x = predictions.index, y = predictions, name = "prediccion"))
fig.update_layout(template = None)

RMSE:  226.3460990479419


In [176]:
X_predict = sm.add_constant(test[exog])
predictions = robust_results.predict(X_predict)
rmse = mean_squared_error(test["pp"], predictions, squared=False)
print("RMSE: ", rmse)
fig = go.Figure(go.Scatter(x = test.index, y = test["pp"], name = "observado"))
fig = fig.add_trace(go.Scatter(x = predictions.index, y = predictions, name = "prediccion"))
fig.update_layout(template = None)

RMSE:  234.7959911397517


In [177]:
X_nov_2022 = sm.add_constant(test[exog])
predictions = robust_results.predict(X_nov_2022[-1:])

In [178]:
predictions

fecha
2022-11-01    2234.389906
dtype: float64